In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
from tf_keras.preprocessing.image import ImageDataGenerator
import tf_keras
from tf_keras.applications import Xception
from tf_keras import layers, models

2024-08-28 22:41:23.474742: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-28 22:41:23.499577: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-28 22:41:23.519466: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-28 22:41:23.524896: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-28 22:41:23.547079: I tensorflow/core/platform/cpu_feature_guar

In [2]:
def load_images_from_folder(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.resize(img, (224, 224))
            images.append(img)
            labels.append(label)
    return np.array(images), np.array(labels)

full_images, full_labels = load_images_from_folder('Full', 0)
half_images, half_labels = load_images_from_folder('Half', 1)
low_images, low_labels = load_images_from_folder('Low', 2)
no_images, no_labels= load_images_from_folder("Absent", 3)

images = np.concatenate((full_images, half_images, low_images, no_images))
labels = np.concatenate((full_labels, half_labels, low_labels, no_labels))
images = images / 255.0

In [3]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train)

In [4]:
base_model = Xception(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

I0000 00:00:1724865100.334578    5796 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724865100.363949    5796 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724865100.364026    5796 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724865100.366855    5796 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724865100.366928    5796 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 128)               262272    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 4)                 516       
                                                                 
Total params: 21124268 (80.58 MB)
Trainable params: 262788 (1.00 MB)
Non-trainable params: 20861480 (79.58 MB)
___________

In [5]:
with tf.device("/GPU:0"):
    history= model.fit(
        datagen.flow(X_train, y_train, batch_size= 32),
        epochs= 5, 
        validation_data= (X_test, y_test)
    )

Epoch 1/5


2024-08-28 22:41:47.122499: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8902
W0000 00:00:1724865107.225718   33991 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865107.394675   33991 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865107.411637   33991 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865107.422505   33991 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865107.434805   33991 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865107.446400   33991 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865107.456112   33991 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865107.467387   33991 gpu_t

 7/15 [=============>................] - ETA: 1s - loss: 0.7671 - accuracy: 0.7188

W0000 00:00:1724865112.055811   33986 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865112.058007   33986 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865112.061010   33986 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865112.062919   33986 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865112.065047   33986 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865112.066825   33986 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865112.068499   33986 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865112.070638   33986 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865112.072816   33986 gp

 9/15 [=================>............] - ETA: 1s - loss: 0.6741 - accuracy: 0.7561

...

15/15 [==============================] - ETA: 0s - loss: 0.4873 - accuracy: 0.8246

W0000 00:00:1724865115.416182   33988 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865115.418584   33988 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865115.420496   33988 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865115.422223   33988 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865115.423830   33988 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865115.426168   33988 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865115.428181   33988 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865115.430259   33988 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865115.431887   33988 gp

15/15 [==============================] - 12s 378ms/step - loss: 0.4873 - accuracy: 0.8246 - val_loss: 0.0923 - val_accuracy: 0.9833
Epoch 2/5


W0000 00:00:1724865116.023866   33988 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865116.026602   33988 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865116.028060   33988 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865116.029607   33988 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865116.031350   33988 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865116.032862   33988 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865116.034765   33988 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865116.036375   33988 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724865116.037894   33988 gp

15/15 [==============================] - 4s 286ms/step - loss: 0.0707 - accuracy: 0.9896 - val_loss: 0.0257 - val_accuracy: 1.0000
Epoch 3/5
15/15 [==============================] - 5s 316ms/step - loss: 0.0328 - accuracy: 0.9979 - val_loss: 0.0114 - val_accuracy: 1.0000
Epoch 4/5
15/15 [==============================] - 4s 289ms/step - loss: 0.0181 - accuracy: 1.0000 - val_loss: 0.0062 - val_accuracy: 1.0000
Epoch 5/5
15/15 [==============================] - 4s 279ms/step - loss: 0.0201 - accuracy: 0.9979 - val_loss: 0.0042 - val_accuracy: 1.0000


In [6]:
model.save("H5/xception.h5")

/home/manankhatri/.local/lib/python3.10/site-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [7]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

4/4 [==============================] - 1s 88ms/step - loss: 0.0042 - accuracy: 1.0000
Test Accuracy: 100.00%
